# Play with hyperas 

I originally ran some simple grid search, but the issue there is that I didn't have a carefull split into training and validation set as it was only to see what performance we could get and which way we should go. 

Apparently---and a bit suprisingly for me---RGB gives us the best performance and so I'll stick with it here. Also we will use the dropout approach for the Bayesian approximation. 

Also we saw that MinMaxScaler is likly best. 

In [1]:
import pandas as pd 

In [2]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

from keras.models import Model
from keras.layers import Input, Dense, Dropout, concatenate, Activation
from keras.optimizers import RMSprop, Adam

from keras.datasets import mnist
from keras.utils import np_utils
from keras.regularizers import l1

Using TensorFlow backend.


In [3]:
import tensorflow as tf
from keras.layers import Dropout, Dense, BatchNormalization, GaussianDropout, GaussianNoise, LeakyReLU
lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.1)
from keras import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
#from livelossplot.keras import PlotLossesCallback
from keras.constraints import MinMaxNorm
from keras.initializers import Constant
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import sys
sys.path.append('../')
from colorml.utils import mapping_to_target_range, read_pickle, get_timestamp_string, plot_predictions, huber_fn, mapping_to_target_range_sig, augment_data
from colorml.descriptornames import * 
from livelossplot.keras import PlotLossesCallback

## Load the data 

In [33]:
 
#metalcenter_descriptors + functionalgroup_descriptors + linker_descriptors + mol_desc  + summed_linker_descriptors + summed_metalcenter_descriptors + summed_functionalgroup_descriptors

In [34]:
def data():
    CHEMICAL_FEATURES = metalcenter_descriptors + functionalgroup_descriptors + linker_descriptors + mol_desc
    df_subset_merged = pd.read_csv('../data/color_feat_merged.csv')
    augment_dict = read_pickle('../data/augment_dict.pkl')
    df_train, df_test = train_test_split(df_subset_merged, train_size=0.7)
    df_train = augment_data(df_train, augment_dict)

    X_train = df_train[CHEMICAL_FEATURES]
    y_train = df_train[['r', 'g', 'b']]

    X_test = df_test[CHEMICAL_FEATURES]
    y_test = df_test[['r', 'g', 'b']]

    name_train = df_train['color_cleaned']
    name_test = df_test['color_cleaned']

    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_test, X_valid, y_test, y_valid, names_valid, names_valid = train_test_split(X_test, y_test, name_test, train_size=0.9)

    y_train = y_train / 255
    y_valid = y_valid / 255
    y_test = y_test / 255
    return X_train, y_train, X_valid, y_valid

In [35]:
def model(X_train, Y_train, X_test, Y_test):
    mlp = Sequential()
    dropout = {{uniform(0, 1)}}
    l1rate = {{uniform(0, 1)}}
    layers = {{choice([
        
        [64, 32, 16, 8], 
        [128, 64, 32, 16, 8], 
        [128, 16, 8], 
        [32, 16, 8],
        [16, 8, 8], 
        [8,8,8,8], 
        [164,64,32,16],
        [164,32,16],
        [164,64,8]]
    )}}
    
    mlp.add(
        Dense(
            layers[0],
            activation="linear",
            kernel_initializer='he_normal',
            input_shape=(178,),
            activity_regularizer=l1(l1rate),
        )
    )
    mlp.add(Activation("relu"))
    mlp.add(Dropout(dropout))

    for layer in layers[1:]:
        mlp.add(
            Dense(
                layer,
                activation="linear",
                kernel_initializer='he_normal',
                activity_regularizer=l1(l1rate),
            )
        )
        mlp.add(Activation("relu"))
        mlp.add(Dropout(dropout))


    mlp.add(
        Dense(3, activation=mapping_to_target_range, kernel_initializer='he_normal')
    )
                    
    mlp.compile(
        optimizer=Adam(learning_rate={{uniform(1e-5, 1e-2)}}),
        loss={{choice([huber_fn, 'mae', 'mse'])}},
        metrics=["mae", "mean_absolute_percentage_error"],
    )
    
    callbacks = []
                    
    
    if {{choice(['es', False])}} == 'es':
        callbacks.append(
                EarlyStopping(
                    monitor="val_loss", patience={{choice([10, 20, 30, 40, 60, 80, 100])}}, verbose=0, mode="auto"
                )
            )
                    
    if {{choice(['lrs', False])}} == 'lrs':
        callbacks.append(
            learning_rate_reduction=ReduceLROnPlateau(
                monitor="val_loss",
                patience={{choice([10, 20, 30, 40, 60, 80, 100])}},
                verbose=1,
                factor={{uniform(0, 1)}},
                min_lr={{uniform(1e-5, 1e-2)}},
            )
        )
                    
    history = mlp.fit(
            X_train,
            y_train,
            callbacks=callbacks,
            epochs=500,
            batch_size={{choice([64, 128, 256])}},
            validation_data=(X_valid, y_valid),
        )
                    
    score, mae, mape = mlp.evaluate(X_valid, y_valid, verbose=0)
                    
    return {'loss': -mae, 'status': STATUS_OK, 'model': model}               
                    
                    
                    

In [ ]:
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='play_w_hyperas')

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Input, Dense, Dropout, concatenate, Activation
except:
    pass

try:
    from keras.optimizers import RMSprop, Adam
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from keras.regularizers import l1
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from keras.layers import Dropout, Dense, BatchNormalization, GaussianDropout, GaussianNoise, LeakyReLU
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau
except:
 